# Dealing with Lookahead Conflicts

This notebook discusses conflicts that have their origin in insufficient looakahead.
We will discuss the following grammar:
```
a : b 'U' 'V'
  | c 'U' 'W'
  
b : 'X'

c : 'X'
```

## Specification of the Scanner

We implement a minimal scanner.  Below we need to declare the token `USELESS`.  The reason is that `Ply` only works when the list `tokens` is defined and contains at least one token.  

In [1]:
import ply.lex as lex

tokens   = [ 'USELESS' ]
literals = ['U', 'V', 'W', 'X']

def t_USELESS(t):
    r'This will never be used.'

def t_error(t):
    r'Unknown token {t.value}.'
    
__file__ = 'main'

lexer = lex.lex()

## Specification of the Parser

In [2]:
import ply.yacc as yacc

We can specify multiple expressions in a single rule.  In this case, we have used the `pass`statement 
as we just want to generate some conflicts. 

In [3]:
def p_a(p):
    """
    a : b 'U' 'V'
      | c 'U' 'W'
    b : 'X'
    c : 'X'   
    """

In [4]:
def p_error(p):
    if p:
        print(f'Syntax error at {p.value}.')
    else:
        print('Syntax error at end of input.')

Setting the optional argument `write_tables` to `False` <B style="color:red">is required</B> to prevent an obscure bug where the parser generator tries  to read an empty parse table.

In [5]:
parser = yacc.yacc(write_tables=False, debug=True)

Generating LALR tables


Let's look at the action table that is generated.  Conflicts are always resolved in favour of shifting.

In [6]:
!cat parser.out

Created by PLY version 3.11 (http://www.dabeaz.com/ply)

Unused terminals:

    USELESS

Grammar

Rule 0     S' -> a
Rule 1     a -> b U V
Rule 2     a -> c U W
Rule 3     b -> X
Rule 4     c -> X

Terminals, with rules where they appear

U                    : 1 2
USELESS              : 
V                    : 1
W                    : 2
X                    : 3 4
error                : 

Nonterminals, with rules where they appear

a                    : 0
b                    : 1
c                    : 2

Parsing method: LALR

state 0

    (0) S' -> . a
    (1) a -> . b U V
    (2) a -> . c U W
    (3) b -> . X
    (4) c -> . X

    X               shift and go to state 4

    a                              shift and go to state 1
    b                              shift and go to state 2
    c                              shift and go to state 3

state 1

    (0) S' -> a .



state 2

    (1) a -> b . U V

    U               shift and go to state 5


state 3

    (2) a -> c . U W

 